### PS7
### Lerong Wang

In [135]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
from sklearn import svm

### (a)

In [136]:
df = pd.read_csv("Auto.csv", na_values="?")
df= df.dropna()
df['mpg_high'] = np.where(df['mpg']>= np.median(df['mpg']), 1, 0)
df['constant'] = 1
df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,constant
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1


In [137]:
X = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = df['mpg_high'].values

In [138]:
k = 4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(X)
MSE_vec_kf = np.zeros(k)
error0 = np.zeros(k)
error1 = np.zeros(k)
k_ind = int(0)

for train_index, test_index in kf.split(X):
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = mean_squared_error(y_pred, y_test)
    error_0 = ((y_pred==0)*(np.abs(y_test-y_pred))).sum() / (y_pred==0).sum()
    error0[k_ind] = (error_0)
    error_1 = ((y_pred==1)*(np.abs(y_test-y_pred))).sum() / (y_pred==1).sum()
    error1[k_ind] = (error_1)
    print(classification_report(y_test, y_pred, digits=3))
    print('MSE of the logistic model is', k_ind, ' is', MSE_vec_kf[k_ind])
    print('error rates for mpg_high = 0 is', error_0)
    print('error rates for mpg_high = 1 is ', error_1)
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('\n')
print('average MSE across the k = 4 test sets is', MSE_kf)
print('average error rates for mpg_high = 0 is', error0.mean())
print('average error rates for mpg_high = 1 is', error1.mean())

k index= 0
             precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

MSE of the logistic model is 0  is 0.0918367346939
error rates for mpg_high = 0 is 0.0576923076923
error rates for mpg_high = 1 is  0.130434782609
k index= 1
             precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

MSE of the logistic model is 1  is 0.102040816327
error rates for mpg_high = 0 is 0.122448979592
error rates for mpg_high = 1 is  0.0816326530612
k index= 2
             precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0.867     0.867        98

MSE of the logistic model is 

### (b)

In [139]:
random_forest = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                       oob_score=True, random_state=25)
random_forest.fit(X, y)
y_pred2 = random_forest.oob_decision_function_
MSE_rf = pd.DataFrame({'ypred' : y_pred2.T[1], 'yvals': y})
MSE_rf['ypred'] = MSE_rf['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)
MSE = mean_squared_error(MSE_rf['yvals'], MSE_rf['ypred'])
MSE_rf1 = MSE_rf[MSE_rf['ypred'] < 0.5]
MSE1 = mean_squared_error(MSE_rf1['yvals'], MSE_rf1['ypred'])
MSE_rf2 = MSE_rf[MSE_rf['ypred'] >= 0.5]
MSE2 = mean_squared_error(MSE_rf2['yvals'], MSE_rf2['ypred'])
print('MSE of the random forest model is:',MSE)
print('error rates for mpg_high = 0 is',MSE1)
print('error rates for mpg_high = 1 is',MSE2)

MSE of the random forest model is: 0.0714285714286
error rates for mpg_high = 0 is 0.0578947368421
error rates for mpg_high = 1 is 0.0841584158416


### (c)

In [140]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(X)
err0 = np.zeros(k)
err1 = np.zeros(k)
svm_mse = np.zeros(k)
k_ind = int(0)

for train_index, test_index in kf.split(X):
    print('k index=', k_ind)
    X_train, X_test = X1[train_index], X1[test_index]
    y_train, y_test = y[train_index], y[test_index]
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    pred_rbf = y_test != y_pred
    mse_rbf = pred_rbf.mean()
    error0_rbf = ((y_pred == 0) * pred_rbf).sum() / (y_pred == 0).sum() 
    error1_rbf = ((y_pred == 1) * pred_rbf).sum() / (y_pred == 1).sum()
    err0[k_ind] = error0_rbf
    err1[k_ind] = error1_rbf
    svm_mse[k_ind] = mse_rbf
    print(classification_report(y_test, y_pred, digits=3)) 
    print('MSE of the svc model is', mse_rbf)
    print('error rates for mpg_high = 0 is', error0_rbf)
    print('error rates for mpg_high = 1 is', error1_rbf)
    k_ind += 1
    
    
MSE_mean = svm_mse.mean()
err0 = err0[~np.isnan(err0)]
err1 = err1[~np.isnan(err1)]
print('\n')
print('average MSE across is', MSE_mean)
print('average error rates for mpg_high = 0 is', err0.mean())
print('average error rates for mpg_high = 1 is', err1.mean())

k index= 0
             precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

MSE of the svc model is 0.540816326531
error rates for mpg_high = 0 is 0.0
error rates for mpg_high = 1 is 0.552083333333
k index= 1
             precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

MSE of the svc model is 0.520408163265
error rates for mpg_high = 0 is 0.520408163265
error rates for mpg_high = 1 is nan
k index= 2
             precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

MSE of the svc model is 0.520408163265
error rates for mpg_high = 0 is 0.53125

### (d)

Average MSE for logistic model is 0.107, Average MSE for random forest model is 0.071, and Average MSE for svc model is 0.507. MSE for random forest model is the smallest. Also, the error rates for mpg_high = 0 and mpg_high = 1 are also relatively low when using random forest model. Thus, the random forest model is the best predictor of mpg_high. 